In [1]:
import os
import csv
import json

import pandas as pd
from sklearn.model_selection import train_test_split

SEED = 42

In [2]:
data_path = '../../data/google_translate.jsonl'
with open(data_path, 'r', encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

In [3]:
data[0]

{'message_tree_id': '001199c7-7135-4e64-bee9-48ea862243b4',
 'tree_state': 'ready_for_export',
 'prompt': {'message_id': '001199c7-7135-4e64-bee9-48ea862243b4',
  'user_id': '42398a03-04b3-4028-9d97-1444708e91cd',
  'created_date': '2023-02-04T18:54:01.048716+00:00',
  'text': 'Do you have any information about the Commodore 64?',
  'role': 'prompter',
  'lang': 'en',
  'review_count': 3,
  'review_result': True,
  'deleted': False,
  'synthetic': False,
  'emojis': {'+1': 8, '_skip_reply': 2},
  'replies': [{'message_id': '314ea243-638d-48d6-9f3d-8edf80f85ec2',
    'parent_id': '001199c7-7135-4e64-bee9-48ea862243b4',
    'user_id': '00bbfff8-0e6a-4e46-93cc-65785d78c8bc',
    'created_date': '2023-02-05T00:38:59.927142+00:00',
    'text': "Of course. The Commodore 64 is a 8-bit home computer that was released by Commodore International in August 1982. It is one of the most successful home computers of all time, with over 17 million units sold. It featured 64 kilobytes of RAM, color gra

In [4]:
def tree_to_pairs(tree, split="train"):
    # get pairs for root prompt and replies
    pairs = message_to_pairs(tree["prompt"], split)
    return pairs


def message_to_pairs(message, split="train"):
    pairs = []
    # check who is the input role (we want only pairs where the input is the prompter)
    input_text = message['translation']
    # check if we have any replies to build pairs from
    if 'replies' in message:
        rank = 0
        for reply in message['replies']:
            if "translation" not in reply:
                continue
            # if reply['role'] == 'prompter':
            #     input_text = reply['translation']
            #     continue
            if 'rank' in reply:
                rank = reply['rank']
            else:
                rank += 1
            # get pairs for current input text ang reply
            pairs.append((input_text, reply['translation'], rank, split, reply['role']))
            # get pairs for reply and its replies
            pairs.extend(message_to_pairs(reply, split))
    return pairs

### Split the trees in train, validation and test sets

In [5]:
data_train, data_test = train_test_split(data, test_size=0.2, random_state=SEED, shuffle=True)
data_val, data_test = train_test_split(data_test, test_size=0.5, random_state=SEED, shuffle=True)

In [6]:
len(data_train), len(data_val), len(data_test)

(6923, 865, 866)

### Get the pairs from the trees

In [7]:
def generate_pairs(data, split="train"):
    pairs = []
    for tree in data:
        pairs += tree_to_pairs(tree, split)
    df = pd.DataFrame(pairs, columns=["prompt", "reply", "rank", "split", "reply role"])
    return df

In [8]:
pairs_train = generate_pairs(data_train, "train")
pairs_val = generate_pairs(data_val, "val")
pairs_test = generate_pairs(data_test, "test")
len(pairs_train), len(pairs_val), len(pairs_test)

(53463, 6684, 6893)

In [9]:
pairs_train = pairs_train[pairs_train["reply role"] == "assistant"].drop(columns=["reply role"])
pairs_val = pairs_val[pairs_val["reply role"] == "assistant"].drop(columns=["reply role"])
pairs_test = pairs_test[pairs_test["reply role"] == "assistant"].drop(columns=["reply role"])
len(pairs_train), len(pairs_val), len(pairs_test)

(37902, 4737, 4875)

In [10]:
pairs_train

,prompt,reply,rank,split
0,"V mojo hišo bodo prispeli 3 prijatelji, rad bi...",Kaj pa igranje lažnivca? Za to igro so potrebn...,0,train
1,"V mojo hišo bodo prispeli 3 prijatelji, rad bi...","Uno: Je klasična igra za otroke in odrasle, pr...",1,train
2,"V mojo hišo bodo prispeli 3 prijatelji, rad bi...",Igrate lahko:\n - 8 noro\n - celina\n - Lažnivec,2,train
3,Napišite hvalnico šolskemu taboru v slogu Mela...,Spomnim se vas prijatelji\nIn spet sva se sreč...,0,train
5,Ali kaj bolj veselega?,♪ Spakirajte in se pridružite zabavi!♪\n♪ Šols...,0,train
...,...,...,...,...
53457,"Ne verjamem, da je ""bres-thh"" pravilno, saj im...","Prav imaš!\nPravilen način reči ""dihaj"" bi bil...",1,train
53458,"Ne verjamem, da je ""bres-thh"" pravilno, saj im...","Žal mi je, da imate prav ""bres-thh"" je napačen...",2,train
53459,"Ne verjamem, da je ""bres-thh"" pravilno, saj im...","Joj. Imaš prav, moja napaka. Beseda ""dih"" je t...",3,train
53460,Nisem materni govorec angleščine in imam težav...,"Vsekakor! Glas ""th"" je podoben ""f"" ali ""v"", le...",2,train


In [11]:
pairs_val

,prompt,reply,rank,split
0,"Prosimo, da vljudno zahtevate ponudbo pri seli...",Dame in gospodje\n\nKmalu se selim iz Nürnberg...,0,val
2,Napišite mi ustrezno zadevo za e-pošto.,Ocena stroškov selitev Nürnberg -> Augsburg; 3...,0,val
4,Katere znamenitosti so v Augsburgu?,Augsburg je zgodovinsko mesto na Bavarskem s š...,1,val
5,Napišite mi ustrezno zadevo za e-pošto.,Možna zadeva je: Zahteva za oceno selitve iz N...,1,val
7,"Prosimo, da vljudno zahtevate ponudbo pri seli...",Dame in gospodje\n\n30. aprila 2023 se namerav...,1,val
...,...,...,...,...
6677,"V redu, zdaj pa poiščite ostre besede, ki se z...","Blaznost, formacija, francoščina, utemeljenost...",2,val
6678,"Poiščite besede esdrújula, ki se začnejo s f","Besede Esdrújula, ki se začnejo s črko ""f"", vk...",1,val
6680,"Poiščite besede esdrújula, ki se začnejo s f","To je nekaj besed esdrújula, ki se začnejo s č...",2,val
6682,Odgovor ni pravilen. Besedi enostavno in plodn...,"Prav imate, poskušali boste izboljšati seznam,...",0,val


In [12]:
pairs_test

,prompt,reply,rank,split
0,Vzemimo naključno matriko $n\krat n$ z realnim...,"Ocenjevanje natančne verjetnosti, da imajo vse...",0,test
2,Hvala vam! Jasno mi je,Vedno z veseljem pomagamo! Vas zanima še kaj?,0,test
3,Hvala vam! Jasno mi je,prosim!\nV veselje mi je bilo pomagati vam. Ve...,1,test
5,Hvala vam! Jasno mi je,Z veseljem pomagam! Ali potrebujete še kaj?,2,test
7,Kakšen je geometrijski pomen izpeljanke?,"Geometrični pomen odvoda je, da pokaže, kako h...",0,test
...,...,...,...,...
6886,"Hvala, bom videl, če lahko naredim te sprememb...",Elektronski seti bobnov so na splošno tišji od...,0,test
6887,"Hvala, bom videl, če lahko naredim te sprememb...","To je odvisno od seta bobnov, ki ga trenutno u...",1,test
6888,"Imam elektronski set bobnov, a vsakič, ko igra...",Pod komplet lahko poskusite postaviti izolacij...,2,test
6890,Kadarkoli poskušam postaviti izolacijski mater...,"Da, obstaja več načinov, kako to popraviti.\n\...",0,test


In [13]:
# combine all splits to a single dataframe
pairs = pd.concat([pairs_train, pairs_val, pairs_test], ignore_index=True)
pairs.head()

,prompt,reply,rank,split
0,"V mojo hišo bodo prispeli 3 prijatelji, rad bi...",Kaj pa igranje lažnivca? Za to igro so potrebn...,0,train
1,"V mojo hišo bodo prispeli 3 prijatelji, rad bi...","Uno: Je klasična igra za otroke in odrasle, pr...",1,train
2,"V mojo hišo bodo prispeli 3 prijatelji, rad bi...",Igrate lahko:\n - 8 noro\n - celina\n - Lažnivec,2,train
3,Napišite hvalnico šolskemu taboru v slogu Mela...,Spomnim se vas prijatelji\nIn spet sva se sreč...,0,train
4,Ali kaj bolj veselega?,♪ Spakirajte in se pridružite zabavi!♪\n♪ Šols...,0,train


In [14]:
pairs.to_csv("../../data/processed/prompt_reply_pairs.csv", index=False, sep=";")

In [15]:
df = pd.read_csv("../../data/processed/prompt_reply_pairs.csv", sep=";")
df

,prompt,reply,rank,split
0,"V mojo hišo bodo prispeli 3 prijatelji, rad bi...",Kaj pa igranje lažnivca? Za to igro so potrebn...,0,train
1,"V mojo hišo bodo prispeli 3 prijatelji, rad bi...","Uno: Je klasična igra za otroke in odrasle, pr...",1,train
2,"V mojo hišo bodo prispeli 3 prijatelji, rad bi...",Igrate lahko:\n - 8 noro\n - celina\n - Lažnivec,2,train
3,Napišite hvalnico šolskemu taboru v slogu Mela...,Spomnim se vas prijatelji\nIn spet sva se sreč...,0,train
4,Ali kaj bolj veselega?,♪ Spakirajte in se pridružite zabavi!♪\n♪ Šols...,0,train
...,...,...,...,...
47509,"Hvala, bom videl, če lahko naredim te sprememb...",Elektronski seti bobnov so na splošno tišji od...,0,test
47510,"Hvala, bom videl, če lahko naredim te sprememb...","To je odvisno od seta bobnov, ki ga trenutno u...",1,test
47511,"Imam elektronski set bobnov, a vsakič, ko igra...",Pod komplet lahko poskusite postaviti izolacij...,2,test
47512,Kadarkoli poskušam postaviti izolacijski mater...,"Da, obstaja več načinov, kako to popraviti.\n\...",0,test
